In [ ]:
!git clone https://github.com/AndreyGuzhov/AudioCLIP.git
%cd AudioCLIP


In [ ]:
!wget https://github.com/AndreyGuzhov/AudioCLIP/releases/download/v0.1/AudioCLIP-Full-Training.pt

In [ ]:
!pip install --upgrade librosa numpy pandas pytorch-ignite scikit-learn scipy termcolor torch torchvision tqdm visdom ftfy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image
import torchvision.transforms as T
import torch
import librosa
import os
from AudioCLIP.model.audioclip import AudioCLIP

#모델 경로 지정
model_path = '/content/AudioCLIP/AudioCLIP-Full-Training.pt'
aclp = AudioCLIP(pretrained=model_path)

print("✅ AudioCLIP model loaded successfully!")


In [ ]:
import torchvision.transforms as T

homonym_data = [
    {
        "text_prompt": "flying animal_bat",
        "image_files": [
            '/content/drive/MyDrive/audioCLIP/image/bat.jpg',
            '/content/drive/MyDrive/audioCLIP/image/bat_animal.jpg'
        ],
        "audio_files": [
            '/content/drive/MyDrive/audioCLIP/audio/bat.mp3',
            '/content/drive/MyDrive/audioCLIP/audio/bat_animal.mp3'
        ]
    },
    {
        "text_prompt": "the flying white crane",
        "image_files": [
            '/content/drive/MyDrive/audioCLIP/image/crane.jfif',
            '/content/drive/MyDrive/audioCLIP/image/crane_animal.jfif'
        ],
        "audio_files": [
            '/content/drive/MyDrive/audioCLIP/audio/crane.mp3',
            '/content/drive/MyDrive/audioCLIP/audio/crane_animal.wav'
        ]
    },
    {
        "text_prompt": "the mouse was broken by angry man",
        "image_files": [
            '/content/drive/MyDrive/audioCLIP/image/mouse.jpg',
            '/content/drive/MyDrive/audioCLIP/image/mouse_animal.jfif'
        ],
        "audio_files": [
            '/content/drive/MyDrive/audioCLIP/audio/mouse.mp3',
            '/content/drive/MyDrive/audioCLIP/audio/mouse_animal.mp3'
        ]
    }
]

image_resolution = aclp.visual.input_resolution
preprocess = T.Compose([
    T.Resize(image_resolution, interpolation=T.InterpolationMode.BICUBIC),
    T.CenterCrop(image_resolution),
    T.ToTensor(),
    T.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
])

In [ ]:
aclp.eval()

# 3개의 동음이의어 데이터에 대해 반복
for data in homonym_data:
    text_label = [data['text_prompt']]
    image_files = data['image_files']
    audio_files = data['audio_files']

    # 결과를 저장할 리스트를 루프마다 초기화
    audio_results = []
    image_results = []

    print(f"\n{'='*20} 분석 시작: '{text_label[0]}' {'='*20}")

    with torch.no_grad():
        # 텍스트 특징은 한 번만 계산
        #텍스트 임베딩 계
        text_features = aclp.encode_text(text_label)
        #임베딩 정규화
        text_features /= torch.linalg.norm(text_features, dim=-1, keepdim=True)

        # (A) 오디오 파일 분석
        for audio_path in audio_files:
            audio, _ = librosa.load(audio_path, sr=44100, mono=True)
            audio_input = torch.from_numpy(audio).reshape(1, 1, -1)
            audio_features = aclp.encode_audio(audio_input)

            #오디오 인코딩
            #임베딩 정규
            audio_features /= torch.linalg.norm(audio_features, dim=-1, keepdim=True)
            #코사인 유사도 구하기
            score = (100.0 * audio_features @ text_features.T).item()
            audio_results.append({'file': os.path.basename(audio_path), 'score': score})



        # (B) 이미지 파일 분석
        for image_path in image_files:
            image = Image.open(image_path).convert("RGB")
            image_input = preprocess(image).unsqueeze(0)
            image_features = aclp.encode_image(image_input)
            image_features /= torch.linalg.norm(image_features, dim=-1, keepdim=True)
            score = (100.0 * image_features @ text_features.T).item()
            image_results.append({'file': os.path.basename(image_path), 'score': score})


    print(f"{text_label}에 대한 각각 점수 출력")
    print(f"오디오: {audio_results}")
    print(f"이미지: {image_results}")
    # 가장 유사한 파일 선택
    best_audio_match = max(audio_results, key=lambda x: x['score'])
    best_image_match = max(image_results, key=lambda x: x['score'])

    # 결과 출력
    print(f"--- [최종 선택] 텍스트 '{text_label[0]}' 와(과) 가장 유사한 파일 ---")
    print(f"가장 유사한 오디오: '{best_audio_match['file']}' (점수: {best_audio_match['score']:.2f})")
    print(f"가장 유사한 이미지: '{best_image_match['file']}' (점수: {best_image_match['score']:.2f})")
    print(f"{'='*55}")

# bat
- 오디오: 정확히 예측
- 이미지 : 정확히 예측
# crane
- 오디오: 정확히 예측, 울음소리가 고정되어 있음
- 이미지: 명확한 이미지가 있음에도 실패
# mouse
- 오디오: 예측 실패
- 이미지: 예측 성공

# 매칭에 실패하는 이유
1. CLIP 모델에서 기존에 수행하던 텍스트 - 이미지 쿼리 성능이 감소
2. 전체 모델 훈련 후 ImageNet의 정확도 40.51% -> 21.79%

#의미를 연결하는 임베딩 기술 서비스
-Sora2와 같은 텍스트 -> 영상 및 이미지 생성


-자동으로 상세한 태그 생성 #자동차 소리, 도로, 강아지


-영상 내 특정 순간 검색

#실습 아이디어

##LLava

1. 특정 이미지를 워드 임베딩 공간에 투영시킨 것과 그 이미지를 나타내는 캡셔닝의 임베딩 유사도 비교
2. 유명 명화를 보여주고 비평가처럼 작품 해설 - '주관적이고 추상적인 상황에서의 대답 유추'

##Minigpt

1. 모호한 이미지(토끼,오리 합친 사진)을 보여주고 모델이 이 이미지를
텍스트 공간에서 어느 쪽으로 가깝게 선형 변환 시키는지 확인하기
2. 평범한 물통을 보여주고 "우주 장치에 들어가는 안테나인데 작동 방식을 설명해줘" 라고 했을 때, 시각적 특징과 완전히 다른 텍스트에 대해 어떤 텍스트를 출력하는지